In [190]:
import requests
import time
import random
import sqlite3
from bs4 import BeautifulSoup
import re
import random


In [207]:
conn = sqlite3.connect('nulled.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS archive_urls
            (original TEXT, mimetype TEXT, timestamp TEXT, endtimestamp TEXT, groupcount integer, uniqcount integer)''')

conn.commit()

In [192]:
import json



# Load the JSON data
with open(r'C:\Users\Aston\Documents\GitHub\web_scraper\nulled.to\archive.json', 'r') as file:
    json_data = json.load(file)

# Insert the data into the SQLite table
for row in json_data:
    cursor.execute('INSERT INTO archive_urls VALUES (?, ?, ?, ?, ?, ?)', row)

conn.commit()


In [193]:
user_agents = [ 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 
    'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148', 
    'Mozilla/5.0 (Linux; Android 11; SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Mobile Safari/537.36' 
]

In [210]:
def create_tables(c):
    cursor = c.cursor()
    # Create a table if it doesn't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS users
                (username TEXT,location text)''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS topics
                (url TEXT,title text,author text,timestamp text,replies text,views text,last_post text)''')
    c.commit()
create_tables(conn)

In [229]:
def scrape_topic(url):
    print(f"scraping: {url[0]}")
    archive_url = f"http://web.archive.org/web/{url[0]}"
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent}
    response = requests.get(archive_url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    topics = soup.find_all("tr",class_="__topic")
    for topic in topics:
        try:
            elements = topic.find_all("td")
            topic_url = elements[1].find("a").get("href")
            title = elements[1].find("a").get("title").split("-")[0].strip()
            timestamp = " ".join(elements[1].find("a").get("title").split("-")[1].split(" ")[3:])

            author = elements[1].find(class_="reverser").text.strip()

            replied = elements[2].text.strip().split("\n")[1]

            views = elements[3].text.strip().split("\n")[0]

            last_post = elements[-1].text.strip().split("\n")[0]

            cursor.execute("INSERT or ignore INTO topics VALUES (?,?, ?, ?, ?, ?, ?)",(topic_url,title, author, timestamp, replied,views,last_post))
            conn.commit()
            cursor.execute("INSERT INTO scraped_urls VALUES (?)",(url[1],))
            conn.commit()
        except Exception as e:
            print(e)
            continue

In [ ]:
scrape_topic(["20190408110906/https://www.nulled.to/forum/3-the-lounge/","test"]) 

In [230]:
def scrape_banlist(url):
    print(f"scraping: {url[0]}")
    archive_url = f"http://web.archive.org/web/{url[0]}"
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent}
    response = requests.get(archive_url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    ban_element = soup.find("tbody",class_='ban-tbody')
    ban_list = soup.find_all("tr")
    for ban in ban_list:
        try:
            elements = ban.find_all("td")
            username = elements[0].text.strip()
            # get image
            location = ban.find("img").get("src").split("/")[-1]

            cursor.execute("INSERT or ignore INTO users VALUES (?,?)",(username,location))
            conn.commit()
            cursor.execute("INSERT INTO scraped_urls VALUES (?)",(url[1],))
            conn.commit()
        except Exception as e:
            print(e):
            continue

In [198]:
scrape_banlist(["20220607002910/https://www.nulled.to/ban-list.php?&st=0","test"]) 

scraping: 20220607002910/https://www.nulled.to/ban-list.php?&st=0


In [236]:
def scrape_award(url):
    print(f"scraping: {url[0]}")
    archive_url = f"http://web.archive.org/web/{url[0]}"
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent}
    response = requests.get(archive_url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    list = soup.find_all("tr")
    for row in list:
        try:
            elements = row.find_all("td")
            username = elements[0].text.strip()
            cursor.execute("INSERT or ignore INTO users VALUES (?)",(username))
            conn.commit()
            cursor.execute("INSERT INTO scraped_urls VALUES (?)",(url[1],))
            conn.commit()
        except Exception as e:
            print(e)
            continue

In [ ]:
scrape_award(["20220124055516/https://www.nulled.to/awards.php?award_id=12","test"]) #works

In [ ]:
cursor.execute('''select endtimestamp ||"/" || original as url, original  from archive_urls left join scraped_urls on scraped_urls.url = original where mimetype = "text/html" and scraped_urls.url is Null ''')
urls = cursor.fetchall()
for url in urls:
    # Check if the url string constain the word forum
    if '/forum/' in url[0].lower():
        scrape_topic(url)

        time.sleep(1)
    elif 'ban-list.php' in url[0].lower():
        scrape_banlist(url)
        time.sleep(1)
    elif 'awards.php' in url[0].lower():
        scrape_award(url)
        time.sleep(1)
    else:
        continue
conn.close()